In [57]:
from crystal_functions.file_readwrite import Crystal_output

class Harmonic(Crystal_output):
    """
    Class Harmonic, inherited from the Crystal_output class, with thermodynamic attributes.
    Used for harmonic phonon calclulations (single file).
    
    Inherited attributes:
        self.edft
        self.nqpoint
        self.qpoint
        self.nmode
        self.frequency
        self.eigenvector
    
    New attributes:
        self.mode, __init__, The list of mode objects.
        self.temperature, __init__, The temperature range for HA thermodynamics.
        self.helmholtz, thermodynamics, Helmholtz free energy of the cell, at HA level.
                        Unit: KJ/mol cell
        * Following attributes are the summed up values of corresponding attributes of
          class Mode.
        self.U_vib, thermodynamics
        self.zp_energy, thermodynamics 
        self.entropy, thermodynamics
        self.C_v, thermodynamics
    """
    def __init__(self, output_name, temperature=[298.15], scelphono=[],
                 write_out=True, filename='HA-thermodynamics.dat'):
        """
        Input:
            output_name, str, Name of the output file.
            Temperature, nTempt * 1 array / list, Temperatures where the thermodynamic
                         properties are computed. Unit: K
            scellphono, ndimension * ndimension list, The supercell manipulation matrix
                        used for vibrational properties. Should be the same as 'SCELPHONO'
                        keyword in CRYSTAL input file.
            write_out, bool, Wheter to print out HA thermodynamic properties.
            filename, str, Name of the printed-out file, used only if write_out = True.
        Output:
            self.temperature, nTempt * 1 array / list, Temperature range. Unit K
            self.mode, nqpoint * nmode list, List of mode objects at all qpoints.
            filename, text file, HA thermodynamic data. 'Thermodynamics' method will be
                      automatically executed and a file will be printed out, if
                      write_out = True.
        """
        import numpy as np

        super(Harmonic, self).read_cry_output(output_name)
        self.temperature = np.array(temperature, dtype=float)
        self.get_mode()
        self.clean_imaginary()

#         vol = self.lattice.volume
        
#         # Transfer the modes in self.freqency into lists of mode objects
#         modes = []
#         for qpoint in self.frequency:
#             qmodes = []
#             for m, freq in enumerate(qpoint):
#                 qmodes.append(Mode(rank=m + 1, frequency=[freq], volume=[vol]))

#             modes.append(qmodes)
        
#         self.mode = modes
        
#         if write_out:
#             self.thermodynamics()
#             wtout = open(filename, 'w')
#             self.print_results(file=wtout)
#             wtout.close()

    def generate_lattice(self, scelphono):
        """
        Generate the pymatgen 'structure' object for the next-step analysis.
        """
        import numpy as np
        from crystal_functions.convert import cry_out2pmg
        import pymatgen
        
        ndimen = self.get_dimensionality()

        if not scelphono or not ndimen:
            self.lattice = cry_out2pmg(self, vacuum=500)
            return self.lattice
        
        scell_mx = np.eye(3, dtype=float)
        scell_mx[: ndimen, : ndimen] = np.array(scelphono)[: ndimen, : ndimen]
        scell_mx = np.linalg.pinv(scell_mx)
        print(scell_mx)

        lattice = cry_out2pmg(self, vacuum=500)
        cell = np.array(lattice.lattice)
        print(np.dot(scell_mx, cell))
        lattice = lattice * [[0.5, 0, 0], [0, 0.5, 0], [0, 0, 1]]

        return lattice

In [58]:
graphene = Harmonic('nostr_modes.out', temperature=[298.15], write_out=False)
graphene.generate_lattice(scelphono=[[12, 0], [0, 12]])

[[0.08333333 0.         0.        ]
 [0.         0.08333333 0.        ]
 [0.         0.         1.        ]]


TypeError: unsupported operand type(s) for *: 'float' and 'Lattice'